In [1]:
#Preamble
import tensorflow as tf
#enable Eager execution
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
import TaylorFlow_190722 as tlf

In [2]:
def cmplxInnerProd(temp,data,psd,df):
    "computes complex inner product in the fourier domain IP = 4 deltaf sum((a * conguagte(b))/Weights)"
    
    weights = tf.sqrt(psd)
    
    return tf.multiply((4*df),tf.reduce_sum(tf.multiply(tf.divide(temp,weights),tf.divide(tf.conj(data),weights))))

def InnerProd(temp,data,psd,df):
    "computes inner product in the fourier domain IP = 4 deltaf RE sum((a * conguagte(b))/Weights)"
    
    return tf.real(cmplxInnerProd(temp,data,psd,df))

def sigsq(temp, psd, df):
    "computes sig^2 = (a|a), which can then be used to normalise function "
    weights = tf.sqrt(psd)
    
    return tf.real(tf.multiply((4*df),tf.reduce_sum(tf.multiply(tf.divide(temp,weights),tf.divide(tf.conj(temp),weights)))))

def sig(temp, psd, df):
    "returns the sigma value of the signal"
    
    return tf.sqrt(sigsq(temp, psd, df))

def TensorNorm(a):
    "divides by max value of tensor to normalise between 0 and 1"
    TensorMax = tf.reduce_max(tf.abs(a))
    Tmax_cmplx_recip = tf.complex(tf.reciprocal(TensorMax),0.)
    
    return tf.multiply(a,Tmax_cmplx_recip),TensorMax

def overlap(temp,data,psd,df):
    "Overlap equation"
    
    norm = tf.divide(1,tf.multiply(sig(temp,psd,df),sig(data,psd,df)))
    inner = InnerProd(temp,data,psd,df)
    overlap = tf.multiply(norm,inner)
        
    return overlap

def SNR(temp,data,psd,df):
    "Returns the SNR along with the normalization"
    norm = tf.reciprocal(sig(temp,psd,df))
    inner = InnerProd(temp,data,psd,df)
    
    return tf.multiply(norm,inner)

In [3]:
m1 = 10
m2 = 10
m3 = 5
m4 = 5
df = 1./512
f_low = 20
f_high = 2000.

# freq1,waveform1 = tlf.TaylorFlow(m1, m2, f_low=f_low, f_high=f_high, df = df, t_c=0., phi_c = 0)
# freq2,waveform2 = tlf.TaylorFlow(m3, m4, f_low=f_low, f_high=f_high, df = df, t_c=0., phi_c = 0)
# n_waveform1,max1 = TensorNorm(waveform1)
# # n_waveform2,max2 = TensorNorm(waveform2)
# n_waveform2 = tf.multiply(waveform2,tf.complex(tf.reciprocal(max1), 0.))



In [4]:
n_waveform1 = np.loadtxt('normpycbc_10sol.txt',dtype= complex).astype(np.complex64)
n_waveform2 = np.loadtxt('normpycbc_5sol.txt',dtype= complex).astype(np.complex64)

In [5]:
psd = tf.complex(np.ones(len(n_waveform1),dtype=np.float32), 0.)

In [6]:
print(n_waveform1.dtype)

complex64


In [7]:
cmplxIP = cmplxInnerProd(n_waveform1, n_waveform2, psd, df)

In [8]:
IP = InnerProd(n_waveform1, n_waveform2, psd, df)

In [9]:
sig1 = sig(n_waveform1, psd, df)

In [10]:
sig2 = sig(n_waveform2, psd, df)

In [11]:
sigsq1= sigsq(n_waveform1, psd, df)

In [12]:
sigsq2= sigsq(n_waveform2, psd, df)

In [13]:
overlap1 = overlap(n_waveform1, n_waveform2, psd, df)

In [14]:
print("CmplxInnrP = %g + %g i" %(tf.real(cmplxIP), tf.imag(cmplxIP)))
print("RealInnerP = %g" %IP)
print("sig1 = %g" %sig1)
print("sig2 = %g" %sig2)
print("sigsq1 = %g" %sigsq1)
print("sigsq2 = %g" %sigsq2)
print("Overlap = %g" %overlap1)

CmplxInnrP = 0.0428467 + 0.0672404 i
RealInnerP = 0.0428467
sig1 = 7.65271
sig2 = 4.34984
sigsq1 = 58.564
sigsq2 = 18.9211
Overlap = 0.00128715
